In [1]:
import pandas as pd
import numpy as np
import requests
import random
from datetime import datetime, timedelta

In [7]:
df_assortment = pd.read_csv('https://raw.githubusercontent.com/AntonMiniazev/Fine_Delivery/main/project_notebooks/assortment_generator.csv?token=GHSAT0AAAAAACNEWBGF77R4TBBOT5V4ARMSZNVLI6Q', dtype=object, delimiter = ';', thousands=',')
df_delivery_types = pd.read_csv('https://raw.githubusercontent.com/AntonMiniazev/Fine_Delivery/main/initial_data/delivery_types.csv?token=GHSAT0AAAAAACNEWBGEENKV6QG33EIMPVA2ZNVLJSQ', dtype=object, delimiter = ';', thousands=',')

### Specifics of orders
1. Create orders for tables Products and Orders
2. Orders should have columns [delivery_date, delivery_type, product_id, order_id,quantity,selling_price, cost_of_sales,zone_id]
Limitations for orders:
- Every product has limits (min and max, max depends on delivery_type: 1 - Bike, 2 - Car)
- Every day has 20-35 orders
- Number of positions for cars 5 to 20 and for bike 2 to 12

In [3]:
# Sets 1 or 2 for a delivery type
def assign_values_within_day(df):
    num_orders = len(df)
    num_ones = int(num_orders * random.uniform(0.6, 0.85))  # Calculate number of 1s

    # Assign 1s and 2s randomly within the day
    values = random.choices([1, 2], k=num_orders)
    values[:num_ones] = [1] * num_ones  # Set the first num_ones elements to 1
    return values

def get_random_positions(delivery_type, assortment = df_assortment):
    if delivery_type == 1:
        num_ids = random.randint(2, 12)  # Bikes will have positions between 2 and 12
    else:
        num_ids = random.randint(5, 20)  # Cars will have positions between 5 and 20
    random_positions = df_assortment['product_id'].sample(n=num_ids).tolist()
    return random_positions

def add_values_to_column(df, values):
    num_rows = len(values)
    num_cols = len(df.columns)
    repeated_df = pd.concat([df] * num_rows, ignore_index=True)
    repeated_df['product_id'] = values[:num_rows]
    return repeated_df

def add_positions(df):
    df_with_positions = pd.DataFrame(columns=df.columns)
    for id in df['order_id']:
        tmp_df = df.query('order_id == @id')
        tmp_df = add_values_to_column(tmp_df,get_random_positions(tmp_df.iloc[0]['delivery_type']))
        df_with_positions = pd.concat([df_with_positions,tmp_df], ignore_index=True)
    return df_with_positions

def add_quantity(row):
    prod_id = row['product_id']
    product_limits = df_assortment.query('product_id == @prod_id')
    if row['delivery_type'] == 1:
        num_q = random.randint(int(product_limits.iloc[0]['limit_min']), int(product_limits.iloc[0]['limit_bike_max']))
    else:
        num_q = random.randint(int(product_limits.iloc[0]['limit_min']), int(product_limits.iloc[0]['limit_car_max']))
    return num_q

def orders_dates(start_date, end_date,df_assortment):
    # Convert start_date and end_date strings to datetime objects
    start_date = datetime.strptime(start_date, "%d.%m.%Y")
    end_date = datetime.strptime(end_date, "%d.%m.%Y")

    # Calculate the number of days between start_date and end_date
    num_days = (end_date - start_date).days + 1

    gen_orders = []
    first_order = 10000  # Initial order_id

    # Generate rows for each date in the range
    for i in range(num_days):
        current_date = start_date + timedelta(days=i)
        num_orders = random.randint(20, 35)  # Random number of orders between 20 and 35
        orders = [(first_order + j, current_date) for j in range(num_orders)]
        gen_orders.extend(orders)
        first_order += num_orders
    
    df = pd.DataFrame(data=gen_orders, columns=['order_id','delivery_date'])
    
    num_orders = len(df)
    zone_ids = [random.randint(1, 5) for _ in range(num_orders)]
    df['zone_id'] = zone_ids
    
    values = []
    for date in df['delivery_date'].unique():
        tmp_df = df.query('delivery_date == @date')
        x = assign_values_within_day(tmp_df)
        values.extend(x)
    df['delivery_type'] = values

    df_with_positions = add_positions(df)
    df_with_positions['quantity'] = df_with_positions.apply(add_quantity,axis=1)
    
    df_with_positions = df_with_positions.merge(df_assortment[['product_id','selling_price','cost_of_sales']], how='left',on='product_id')
    
    df_with_positions['total_price'] = df_with_positions['quantity'] * df_with_positions['selling_price'].astype('float16')
    df_with_positions['total_cost'] = df_with_positions['quantity'] * df_with_positions['cost_of_sales'].astype('float16')
    
    df_with_positions['delivery_date'] = pd.to_datetime(df_with_positions['delivery_date'],format="%d.%m.%Y").dt.date    
    
    return df_with_positions   

In [5]:
df_orders = orders_dates("01.04.2023", "15.04.2023",df_assortment)

In [6]:
df_orders

,order_id,delivery_date,zone_id,delivery_type,product_id,quantity,selling_price,cost_of_sales,total_price,total_cost
0,10000,2023-04-01,4,1,100290,1,350,234.5,350.0,234.50000
1,10000,2023-04-01,4,1,100190,2,250,180,500.0,360.00000
2,10000,2023-04-01,4,1,100973,2,500,355,1000.0,710.00000
3,10001,2023-04-01,4,1,101002,2,575,402.5,1150.0,805.00000
4,10001,2023-04-01,4,1,100276,4,40,27.6,160.0,110.37500
...,...,...,...,...,...,...,...,...,...,...
3155,10404,2023-04-15,3,2,100276,4,40,27.6,160.0,110.37500
3156,10404,2023-04-15,3,2,101114,1,1100,803,1100.0,803.00000
3157,10404,2023-04-15,3,2,100927,4,180,135,720.0,540.00000
3158,10405,2023-04-15,5,1,101035,1,900,666,900.0,666.00000


In [8]:
final_orders = df_orders.groupby(['order_id','delivery_date','delivery_type','zone_id']).agg({'total_price':'sum'}).reset_index().rename(columns={'total_price':'total_value'})

In [9]:
final_products = df_orders[['product_id'
                                     ,'order_id'
                                     ,'quantity'
                                     ,'selling_price'
                                     ,'cost_of_sales'
                                     ,'total_price'
                                     ,'total_cost']]

In [7]:
final_orders.to_csv('orders.csv', index=False, sep=";")
final_products.to_csv('products.csv', index=False,sep=";")

NameError: name 'final_orders' is not defined